In [ ]:
!export CUDA_VISIBLE_DEVICES=3

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def sound( x, rate=8000, label=''):
    from IPython.display import display, Audio, HTML
    if label is '':
        display( Audio( x, rate=rate))
    else:
        display( HTML( 
        '<style> table, th, td {border: 0px; }</style> <table><tr><td>' + label + 
        '</td><td>' + Audio( x, rate=rate)._repr_html_()[3:] + '</td></tr></table>'
        ))
import pdb

In [ ]:
from wham_data_utils import static_data_prep, create_wham_tse_csv
import os
import torch
import numpy as np
from speechbrain.dataio.batch import PaddedBatch

In [ ]:
def listen_batch(batch):
    mix = batch['mix_sig'].data
    s1 = batch['s1_sig'].data
    s2 = batch['s2_sig'].data
    noise = batch['noise_sig'].data
    enr = batch['enr_sig'].data
    diff = mix - s1 - s2 - noise
    print(abs(mix).max())
    print((diff**2).mean())
    sound(mix[0].numpy(), rate=hparams['sample_rate'], label='mix')
    sound(s1[0].numpy(), rate=hparams['sample_rate'], label='s1')
    sound(s2[0].numpy(), rate=hparams['sample_rate'], label='s2')
    sound(enr[0].numpy(), rate=hparams['sample_rate'], label='enr')

# New version 

In [ ]:
hparams = {
    'train_dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'valid_dataloader_opts': {
        'batch_size': 1,
        'num_workers': 0,
    },
    'base_folder_dm_info_list': [
        {
            'path': '/mnt/data1/wsj0.8k/si_tr_s/',
            'ext': 'wav',
            'type': 'clean',
        }
    ],
    'data_clean_prob': 1.,
    'wsj_folder': '/mnt/data1/wsj0.8k',
    'data_folder': '/mnt/data1/wham/wham_original',
    'train_txtpath': '/mnt/data1/Speech/wsj_tse/mix_2_spk_tr_extr.txt',
    'valid_txtpath': '/mnt/data1/Speech/wsj_tse/mix_2_spk_tt_extr.txt',
    'train_data': './wham_tse_tr.csv',
    'valid_data': './wham_tse_tt.csv',
    'sample_rate': 8000,
    'training_signal_len': 24000,
}

In [ ]:
create_wham_tse_csv(os.path.join(hparams['data_folder'], 'wav8k/min/tt'),
                    hparams['wsj_folder'], hparams['valid_txtpath'], hparams['valid_data'])

In [ ]:
new_ds = static_data_prep(hparams, 'valid')
new_dl = torch.utils.data.DataLoader(
        new_ds,
        batch_size=hparams["valid_dataloader_opts"]["batch_size"],
        num_workers=hparams["valid_dataloader_opts"]["num_workers"],
        collate_fn=PaddedBatch,
        worker_init_fn=lambda x: np.random.seed(
            int.from_bytes(os.urandom(4), "little") + x
        ),
    )

In [ ]:
for i, new_batch in enumerate(new_dl):  
    if i == 2:
        listen_batch(new_batch)
        break

In [ ]:
from wham_data_utils import dynamic_mixing_prep

create_wham_tse_csv(os.path.join(hparams['data_folder'], 'wav8k/min/tr'),
                    hparams['wsj_folder'], hparams['train_txtpath'], hparams['train_data'])
dl = dynamic_mixing_prep(hparams, 'train')
for i, new_batch in enumerate(dl):  
    if i == 2:
        listen_batch(new_batch)
        break